In [1]:
from __future__ import annotations

import json
import cProfile
import pstats

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
data_source = DataSource(
    rec=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [5]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [7]:
with cProfile.Profile() as pr:
  rec = data_source.load(data)
  db = data_source.db
  
  stats = pstats.Stats(pr)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\lworm\AppData\Local\Temp\ipykernel_23952\1061344892.py:2 in <module>                    │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                               │
│ ❱ 2   rec = data_source.load(data)                                                               │
│   3   db = data_source.db                                                                        │
│   4                                                                                              │
│   5   stats = pstats.Stats(pr)                                                                   │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\importing.py:510 in load                     │
│                                                                                                  │
│   507 │   │   py_cache = {}                                                                      │
│   508 │   │   db_cache = self.db if self.cache is not False else None                            │
│   509 │   │                                                                                      │
│ ❱ 510 │   │   rec = _map_record(self.rec, self, input_data, py_cache, db_cache)                  │
│   511 │   │                                                                                      │
│   512 │   │   if db_cache is not None:                                                           │
│   513 │   │   │   db_cache[type(rec)] |= rec                                                     │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\importing.py:433 in _map_record              │
│                                                                                                  │
│   430 │   │   target_type = rel.record_type                                                      │
│   431 │   │                                                                                      │
│   432 │   │   for sub_path, sub_data in sub_data.items():                                        │
│ ❱ 433 │   │   │   target_rec = _map_record(                                                      │
│   434 │   │   │   │   target_type,                                                               │
│   435 │   │   │   │   target_map,                                                                │
│   436 │   │   │   │   sub_data,                                                                  │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\importing.py:415 in _map_record              │
│                                                                                                  │
│   412 │   │   return py_cache[rec_type][rec._index]                                              │
│   413 │                                                                                          │
│   414 │   if db_cache is not None:                                                               │
│ ❱ 415 │   │   recs = db_cache[rec_type][[rec._index]].load()                                     │
│   416 │   │   if len(recs) == 1:                                                                 │
│   417 │   │   │   rec = next(iter(recs.values()))                                                │
│   418 │   │   │   py_cache[rec_type][rec._index] = rec                                           │
│                                                                                                  │
│ C:\Users\lworm\Repos\py-research\src\py_research\db\base.py

In [10]:
rec

{'term': 'test', 'result_count': 3, 'results': [{'number': 0, 'name': 'baking cake', 'start': '2020-01-01', 'end': '2020-01-04', 'status': 'done', 'org': {'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville', 'projects': <LoadStatus.unloaded: 1>, '_id': UUID('279d4651-5424-4139-9c14-cc67b9cc50d5')}, 'tasks': [{'name': <LoadStatus.unloaded: 1>, 'project': <LoadStatus.unloaded: 1>, 'assignees': [{'name': 'John', 'age': <LoadStatus.unloaded: 1>, 'tasks': <LoadStatus.unloaded: 1>, '_id': UUID('9b2a36bd-99af-45ed-9cb2-39792c4e1751')}], 'status': 'todo', '_id': UUID('1b37a53f-0d90-4246-95d0-bb3969f743ab')}, {'name': 'task2', 'project': <LoadStatus.unloaded: 1>, 'assignees': [{'name': 'John', 'age': <LoadStatus.unloaded: 1>, 'tasks': <LoadStatus.unloaded: 1>, '_id': UUID('c7c4e663-bcdc-433e-9c9f-765b694b22c5')}], 'status': 'todo', '_id': UUID('f9a6b04a-0c48-4326-9879-c0cb4c521116')}, {'name': 'task3', 'project': <LoadStatus.unloaded: 1>, 'assignees': [{'name': 'Jane', 'age': <Lo

In [11]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         455454 function calls (377858 primitive calls) in 0.356 seconds

   Ordered by: cumulative time
   List reduced from 190 to 47 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      386    0.000    0.000    0.356    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:1869(__hash__)
     26/1    0.001    0.000    0.356    0.356 C:\Users\lworm\Repos\py-research\src\py_research\db\importing.py:370(_map_record)
42978/386    0.088    0.000    0.338    0.001 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:33(gen_int_hash)
11952/4386    0.007    0.000    0.323    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:63(<genexpr>)
10571/4000    0.006    0.000    0.321    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:27(_hash_sequence)
20270/8000    0.031    0.000    0.314    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:29(<lambda>)
       26    0.00